In [ ]:
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import os

# Generate Train Data
train_files = os.listdir('/kaggle/input/opencode-23-kaggle-competition/train/Regression')
train_data = []

for file in train_files:
    if file.endswith('.jpg'):
        img_id = file.split('.')[0]
        target = int(img_id.split('_')[0])
        train_data.append({'id': img_id, 'target': target})

train_df = pd.DataFrame(train_data)

# Load images and preprocess
train_images = []
train_targets = []

for idx, row in train_df.iterrows():
    img_path = f"/kaggle/input/opencode-23-kaggle-competition/train/Regression/{row['id']}.jpg"
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))  # Resize to 224x224 for MobileNet
    train_images.append(img)
    train_targets.append(row['target'])

train_images = np.array(train_images) / 255.0  # Normalize
train_targets = np.array(train_targets)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_targets, test_size=0.2, random_state=42)

# Load MobileNet base model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build model on top of MobileNet
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3)

# Train model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32, callbacks=[early_stop])


In [ ]:
test_files = os.listdir('/kaggle/input/opencode-23-kaggle-competition/test/Regression')
test_data = [file.split('.')[0] for file in test_files if file.endswith('.jpg')]
test_df = pd.DataFrame(test_data, columns=['id'])

# Predict
test_images = []
for idx, row in test_df.iterrows():
    img_path = f"/kaggle/input/opencode-23-kaggle-competition/test/Regression/{row['id']}.jpg"
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    test_images.append(img)

test_images = np.array(test_images) / 255.0

predictions = model.predict(test_images).flatten()

# Create submission file
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'target': predictions
})

submission_df.to_csv('/kaggle/working/submission_2.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import os

# Generate Train Data
train_files = os.listdir('/kaggle/input/opencode-23-kaggle-competition/train/Regression')
train_data = []

for file in train_files:
    if file.endswith('.jpg'):
        img_id = file.split('.')[0]
        target = int(img_id.split('_')[0])
        train_data.append({'id': img_id, 'target': target})

train_df = pd.DataFrame(train_data)

# Load images and preprocess
train_images = []
train_targets = []

for idx, row in train_df.iterrows():
    img_path = f"/kaggle/input/opencode-23-kaggle-competition/train/Regression/{row['id']}.jpg"
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))  # Resize to 224x224 for MobileNet
    train_images.append(img)
    train_targets.append(row['target'])

train_images = np.array(train_images) / 255.0  # Normalize
train_targets = np.array(train_targets)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_targets, test_size=0.2, random_state=42)

# Load MobileNet base model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build model on top of MobileNet
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1)
])

# Create an Adam optimizer instance with specified learning rate
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='mse')

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5)

# Train model with data augmentation
batch_size = 32
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)
steps_per_epoch = X_train.shape[0] // batch_size

model.fit(
    train_generator,
    validation_data=(X_val, y_val),
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    callbacks=[early_stop]
)

In [ ]:
test_files = os.listdir('/kaggle/input/opencode-23-kaggle-competition/test/Regression')
test_data = [file.split('.')[0] for file in test_files if file.endswith('.jpg')]
test_df = pd.DataFrame(test_data, columns=['id'])

# Predict
test_images = []
for idx, row in test_df.iterrows():
    img_path = f"/kaggle/input/opencode-23-kaggle-competition/test/Regression/{row['id']}.jpg"
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    test_images.append(img)

test_images = np.array(test_images) / 255.0

predictions = model.predict(test_images).flatten()

# Create submission file
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'target': predictions
})

submission_df.to_csv('/kaggle/working/submission.csv', index=False)